In [85]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.by import By
import getpass
import time

In [92]:
# initialize browser
browser = webdriver.Firefox()

In [93]:
# set a time to wait before every DOM item retrieval
browser.implicitly_wait(5)

In [94]:
# navigate to facebook
browser.get('http://www.facebook.com')

In [95]:
# enter login cridentials and login
# skip if already logged in
username_input_box = browser.find_element_by_id("email")
password_input_box = browser.find_element_by_id("pass")
submit_button  = browser.find_element_by_id("loginbutton")
email = input("Enter email: ")
password = getpass.getpass("Enter password: ")
username_input_box.send_keys(email)
password_input_box.send_keys(password)
submit_button.click()

Enter email: awaaab10@gmail.com
Enter password: ········


In [96]:
def wait_get_element(browser, wait, by_type, by_value):
    try:
        element = WebDriverWait(browser, wait).until(
            EC.presence_of_element_located((by_type, by_value))
        )
    except TimeoutException:
        print(f'TIME OUT, WAITED FOR {wait}s for element {by_value}')
        return None
    return element

In [97]:
# navigate to facebook, enter and search text through facebook search bar
# browser.get('http://www.facebook.com')

search_box = wait_get_element(browser, 10, By.XPATH, '//input[@aria-label="Search Facebook"]')
if search_box:
    search_text = input("Enter search text: ")
    search_box.send_keys(search_text)
    search_box.send_keys(Keys.RETURN)
    search_box.send_keys(Keys.RETURN)

search_posts_option = wait_get_element(browser, 3, By.XPATH, '//span[text()="Posts"]')
if search_posts_option:
    search_posts_option.click()

search_filter =  wait_get_element(browser, 3, By.XPATH, '//span[text()="Posts From"]')
if search_filter:
    search_filter.click()

filter_allow_only_public = wait_get_element(browser, 10, By.XPATH, '//span[text()="Public Posts"]/..')
if filter_allow_only_public:
    browser.execute_script("arguments[0].click();", filter_allow_only_public)
    print("DONE")

Enter search text: kendrick lamar
DONE


In [102]:
results_file = open("results.txt", "w")
i = 1
max_posts = 20
scroll_wait_time = 30
retry_wait_time = 30
retry = False
while i <= max_posts:
    ith_post_xpath = f'/html/body/div[1]/div/div/div[1]/div[3]/div/div/div[1]/div/div[2]/div/div/div/div/div/div/div[{i}]/div/div/div/div/div[3]/a/div[1]/div[1]/span/div'
    try:
        element = browser.find_element(By.XPATH, ith_post_xpath)
    except NoSuchElementException:
        if not retry:
            browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        element = wait_get_element(browser, scroll_wait_time, By.XPATH, ith_post_xpath)
    if element:
        results_file.write(element.text)
        results_file.write('\n')
        retry = False
        i += 1
    else:
        print(f'Failed to retrieve {i}th post. Retrying..')
        retry = True
        time.sleep(retry_wait_time)
results_file.close()
print("DONE")

DONE
